In [2]:
#CLASSIFICATION

from ultralytics import YOLO
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import os

# 1. Reload your trained weights
weights = "fold_1/results/weights/best.pt"
model   = YOLO(weights)

# 2. Grab the model's own names dict
#    e.g. {0: "crack_issues", 1: "potholes"}
names = model.names  

# 3. Validation root and your folder names
val_root = "Road_Damage/test"
folders  = ["potholes", "crack_issues", "alligator_crack_issues", "ravelling", "open_manhole"]  # whatever sub-folders you have

# 4. Invert names → index for easy lookup
name_to_idx = {v: k for k, v in names.items()}

y_true, y_pred = [], []

for folder in folders:
    true_idx = name_to_idx[folder]
    img_dir  = os.path.join(val_root, folder, "images")
    for img in os.listdir(img_dir):
        if not img.lower().endswith((".jpg", ".jpeg", ".png")):
            continue
        path = os.path.join(img_dir, img)
        y_true.append(true_idx)
        res = model(path, imgsz=640)[0]
        y_pred.append(int(res.probs.top1))

# 5. Print classification report
print("\n📊 Classification Report:")
print(classification_report(
    y_true, 
    y_pred, 
    target_names=[names[i] for i in sorted(name_to_idx.values())],
    zero_division=0
))

# 5. Build & save the confusion matrix
cm   = confusion_matrix(y_true, y_pred, labels=sorted(name_to_idx.values()))
disp = ConfusionMatrixDisplay(cm,
       display_labels=[names[i] for i in sorted(name_to_idx.values())])

fig, ax = plt.subplots(figsize=(7,6))
disp.plot(ax=ax, cmap="Blues", xticks_rotation="vertical")
plt.title("Validation Confusion Matrix")
plt.tight_layout()

# **Save** to file instead of (or as well as) plt.show()
out_path = "confusion_matrix.png"
plt.savefig(out_path, dpi=150)
print(f"Saved confusion matrix to {out_path}")
plt.close(fig)


image 1/1 C:\Users\jazzb\ImageDetection-Yolov9\Road_Damage\val\potholes\images\01 - Chuck HolePunch Out (6).jpg: 640x640 potholes 1.00, crack_issues 0.00, open_manhole 0.00, pumping_and_depression 0.00, alligator_crack_issues 0.00, 7.4ms
Speed: 6.3ms preprocess, 7.4ms inference, 0.1ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 C:\Users\jazzb\ImageDetection-Yolov9\Road_Damage\val\potholes\images\01 - Chuck-holePunch-out (RL).jpg: 640x640 potholes 1.00, crack_issues 0.00, pumping_and_depression 0.00, open_manhole 0.00, ravelling 0.00, 11.1ms
Speed: 7.4ms preprocess, 11.1ms inference, 0.1ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 C:\Users\jazzb\ImageDetection-Yolov9\Road_Damage\val\potholes\images\01 - Patholes.jpg: 640x640 ravelling 0.95, crack_issues 0.03, pumping_and_depression 0.02, potholes 0.01, alligator_crack_issues 0.00, 5.7ms
Speed: 7.1ms preprocess, 5.7ms inference, 0.1ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 C:\Users\jaz